This is to demonstrate how to relaunch failed jobs given the name of the deployment (e.g. ```Arabia_14```).

In [1]:
import hyp3_sdk

# Specify deployment URL (uncomment one of the lines below as appropriate)
#deploy_url = 'https://hyp3-tibet.asf.alaska.edu' #for Tibet
#deploy_url = 'https://hyp3-isce.asf.alaska.edu' #for access
deploy_url = 'https://hyp3-nisar-jpl.asf.alaska.edu' #for access

In [14]:
# Define job-name
# [14, 21, 28, 43, 72, 87, 94, 101, 116, 123, 130, 145, 160, 174]
job_name = 'Arabia_21'

hyp3_isce = hyp3_sdk.HyP3(deploy_url)

jobs = hyp3_isce.find_jobs(name=job_name)
print(jobs)

2638 HyP3 Jobs: 972 succeeded, 391 failed, 0 running, 1275 pending.


In [22]:
jobs = [14, 21, 28, 43, 72, 87, 94, 101, 116, 123, 130, 145, 160, 174]

for i in jobs:
    job_name = f'Arabia_{i}'
    print('\033[1m' + job_name)
    jobs = hyp3_isce.find_jobs(name=job_name)
    print('\033[0m', jobs)

Arabia_14


ReadTimeout: HTTPSConnectionPool(host='hyp3-nisar-jpl.asf.alaska.edu', port=443): Read timed out. (read timeout=None)

In [11]:
# parse jobs
pending_jobs = hyp3_isce.find_jobs(status_code='PENDING') + \
               hyp3_isce.find_jobs(status_code='RUNNING') + \
               hyp3_isce.find_jobs(status_code='SUCCEEDED')

failed_jobs = hyp3_isce.find_jobs(status_code='FAILED', name=job_name)

In [25]:
jobs = hyp3_isce.find_jobs()
print(jobs)

29341 HyP3 Jobs: 9692 succeeded, 3 failed, 159 running, 19487 pending.


In [26]:
(8076 / 4) 

2019.0

In [3]:
# restructure dictionaries to make parsing easier
pending_jobs = [{'name': r.name,
              'job_type': r.job_type,
              'job_parameters': {'granules': r.job_parameters['granules'],
                                 'secondary_granules': r.job_parameters['secondary_granules']}} 
             # NOTE THERE IS AN INDEX - this is to submit only a subset of Jobs
             for r in pending_jobs]

failed_jobs = [{'name': r.name,
              'job_type': r.job_type,
              'logs': r.logs,
              'job_parameters': {'granules': r.job_parameters['granules'],
                                 'secondary_granules': r.job_parameters['secondary_granules']}} 
             # NOTE THERE IS AN INDEX - this is to submit only a subset of Jobs
             for r in failed_jobs]

Ensure only unprocessed failed scenes are passed

In [4]:
# Remove duplicate failed jobs
done = []
result = []
for d in failed_jobs:
    if d['job_parameters'] not in done:
        done.append(d['job_parameters'])  # note it down for further iterations
        result.append(d)
print("We are removing {} duplicate failed jobs for submission".format(len(failed_jobs) - len(result)))
failed_jobs = result

# Remove duplicate pending/running/succeeded jobs
done = []
result = []
for d in pending_jobs:
    if d['job_parameters'] not in done:
        done.append(d['job_parameters'])  # note it down for further iterations
for d in failed_jobs:
    if d['job_parameters'] not in done:
        result.append(d)
print("We are removing {} failed jobs for submission that are being/have been processed".format(len(failed_jobs) - len(result)))
failed_jobs = result

We are removing 0 duplicate failed jobs for submission
We are removing 0 failed jobs for submission that are being/have been processed


Print basic info/logs for each failed job that needs to be rerun

In [5]:
# query failed jobs and print URLs for log output
for job in failed_jobs:
    print('Job log:', job['logs'])

In [6]:
failed_jobs[0:2]

[]

Re-query the job to check progress

In [7]:
prepared_jobs = []
for d in failed_jobs:
    d.pop('logs', None)
    prepared_jobs.append(d)

In [130]:
#UNCOMMENT TO SUBMIT
#submitted_jobs = hyp3_sdk.Batch()
#for batch in hyp3_sdk.util.chunk(prepared_jobs):
#    submitted_jobs += hyp3_isce.submit_prepared_jobs(batch)

Re-query the job to check progress

In [131]:
jobs_rerun = hyp3_isce.find_jobs(name=job_name)
print(jobs_rerun)

2639 HyP3 Jobs: 1236 succeeded, 737 failed, 270 running, 396 pending.


Track job progress

In [132]:
# parse jobs
updated_jobs = hyp3_isce.find_jobs(status_code='PENDING') + \
               hyp3_isce.find_jobs(status_code='RUNNING') + \
               hyp3_isce.find_jobs(status_code='SUCCEEDED')

# restructure dictionaries to make parsing easier
updated_jobs = [{'name': r.name,
              'job_type': r.job_type,
              'job_parameters': {'granules': r.job_parameters['granules'],
                                 'secondary_granules': r.job_parameters['secondary_granules']}} 
             # NOTE THERE IS AN INDEX - this is to submit only a subset of Jobs
             for r in updated_jobs]

# Find jobs that failed again
done = []
result = []
for d in updated_jobs:
    if d['job_parameters'] not in done:
        done.append(d['job_parameters'])  # note it down for further iterations
for d in failed_jobs:
    if d['job_parameters'] not in done:
        result.append(d)
print("There are {} jobs that failed again after reprocessing".format(len(result)))

There are 0 jobs that failed again after reprocessing


In [133]:
print("See each failed job printed below:")
for d in result:
    print(d)

See each failed job printed below:
